In [1]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv('./뭉티기test1.csv',encoding = 'utf-8-sig')
print(df)
df.describe()

     Unnamed: 0          상품번호  랭킹                                상품명  \
0             1  6.660420e+09   1  당일도축 대구뭉티기 육사시미 한우 육회 생고기 뭉텅이 뭉태기   
1             2  6.660420e+09   1  당일도축 대구뭉티기 육사시미 한우 육회 생고기 뭉텅이 뭉태기   
2             3  6.660420e+09   1  당일도축 대구뭉티기 육사시미 한우 육회 생고기 뭉텅이 뭉태기   
3             4  6.660420e+09   1  당일도축 대구뭉티기 육사시미 한우 육회 생고기 뭉텅이 뭉태기   
4             5  6.660420e+09   1  당일도축 대구뭉티기 육사시미 한우 육회 생고기 뭉텅이 뭉태기   
..          ...           ...  ..                                ...   
206         207  6.943181e+09  74          대구 뭉티기 소고기 국거리 초기이유식 육회고기   
207         208  6.943181e+09  74          대구 뭉티기 소고기 국거리 초기이유식 육회고기   
208         209           NaN  76                대구 뭉티기 우둔살 250g 덩어리   
209         210           NaN  78                    1등급 한우 뭉티기 200g   
210         211           NaN  80            (대정한우) 냉장 한우 뭉티기 200g[]   

                               옵션     가격    리뷰  구매건수  찜하기  스마트스토어    쇼핑몰  \
0                     당일도축 육회200g  11900  28.0  65.0   34  

,Unnamed: 0,상품번호,랭킹,가격,리뷰,구매건수,찜하기,스마트스토어,배송비
count,211.00000,1.910000e+02,211.000000,211.000000,206.000000,184.000000,211.000000,211.000000,211.000000
mean,106.00000,5.585390e+09,28.431280,28461.753555,694.932039,503.467391,204.682464,0.905213,3239.336493
std,61.05462,6.164387e+08,20.709528,25720.604660,1540.940396,877.821695,328.287474,0.293617,1203.493936
min,1.00000,4.762994e+09,1.000000,8500.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,53.50000,5.171622e+09,10.000000,19500.000000,25.250000,40.750000,12.000000,1.000000,3000.000000
50%,106.00000,5.305004e+09,25.000000,23400.000000,111.000000,70.000000,37.000000,1.000000,3500.000000
75%,158.50000,6.039069e+09,46.000000,34450.000000,813.500000,638.000000,303.000000,1.000000,4000.000000
max,211.00000,6.989547e+09,80.000000,288000.000000,7740.000000,4030.000000,1527.000000,1.000000,4000.000000


In [2]:
# 열 정리
df = df.drop(['Unnamed: 0','용도','부위','등급'], axis=1)

In [3]:
total  = df['랭킹'].nunique()

In [4]:
df['중량'].unique()

array(['200g', '250g', '300g', '300~600g', nan, '150g', '600g',
       '120g, 240g', '120~240g', '120g', '300gg', '250,600g', '500g',
       '국거리300g+다짐육300gg'], dtype=object)

### 중량처리
아이디어
1. 중량 g 떼기 및 표준 형식 ***g 아닌 값 0으로 설정
2. 옵션에 g을 찾아 중량 입력, 수정
3. 없다면 상품명에서 g을 찾아 중량 입력 
// kg은 사전에 제외함

In [5]:
# 결측치 처리
df['중량'] = df['중량'].replace(np.nan,0)

# 중량 열 type 맞추기
def weight_setting(x):
    if type(x) == int:
        return x
    elif len(x) >4:
        return 0
    else:
        return int(x[0:-1])

df['중량'] = df['중량'].map(lambda x: weight_setting(x))
print(df['중량'])

0      200
1      200
2      200
3      200
4      200
      ... 
206      0
207      0
208      0
209      0
210      0
Name: 중량, Length: 211, dtype: int64


In [6]:
# 중량처리

for i in df.index: # 행 반복
    new = 0 
    option = str(df['옵션'][i])
    product_name = str(df['상품명'][i])
    # 옵션에 g이 있을 경우
    if 'g' in option:
        if 'kg' not in option:
            index = -1
            while True:
                index = option.find('g', index + 1)
                if index == -1:
                    break
                else:
                    new += int(option[index-3:index])
    # 옵션에 g가 없을 경우 상품명에서 찾아본다
    elif 'g' in product_name:
        index = product_name.find('g')
        new = product_name[index-3:index]
    else:
        pass

    if new == int(df['중량'][i]):
        pass
    else:
        df.loc[i,'중량'] = new

print(df.head(15))

ValueError: invalid literal for int() with base 10: '.5k'

In [4]:
# 중량 100g당 가격
df['중량'] = df['중량'].map(lambda x: int(x[:-1]))
print(df['중량'])

0      200
1      220
2      220
3      300
4      600
      ... 
159    300
160    300
161    300
162    500
163    300
Name: 중량, Length: 164, dtype: int64


In [5]:
df['100g당_가격'] = 0
df['100g당_가격'] =  (df['가격'] / df['중량']) * 100
df['100g당_가격'] = df['100g당_가격'].map(lambda x : math.ceil(x))
print(df['100g당_가격'])

0      12450
1       5864
2       5864
3       6000
4       6000
       ...  
159     6967
160     6634
161     6967
162     6180
163     6000
Name: 100g당_가격, Length: 164, dtype: int64


In [6]:
# 300g 미만인 상품의 100g당 가격 분석

df[df['중량'] <300]['100g당_가격'].describe()

count       57.000000
mean      8794.649123
std       2337.449598
min       4360.000000
25%       7050.000000
50%       8400.000000
75%      10000.000000
max      14450.000000
Name: 100g당_가격, dtype: float64

In [7]:
df.index +=1
df.reset_index(inplace=True)
df.rename(columns={'index':'key'},inplace=True)

In [8]:
df['배송비포함_가격'] = 0
df['배송비포함_가격'] =  (df['가격'] + df['배송비'])
print(df['배송비포함_가격'])


0      24900
1      16400
2      16400
3      21000
4      39000
       ...  
159    24400
160    23400
161    23900
162    33900
163    22000
Name: 배송비포함_가격, Length: 164, dtype: int64


In [9]:
df['100g당_배송비포함_가격'] = 0
df['100g당_배송비포함_가격'] =  (df['배송비포함_가격'] / df['중량']) * 100
df['100g당_배송비포함_가격'] = df['100g당_배송비포함_가격'].map(lambda x : math.ceil(x))
print(df['100g당_배송비포함_가격'])

0      12450
1       7455
2       7455
3       7000
4       6500
       ...  
159     8134
160     7800
161     7967
162     6780
163     7334
Name: 100g당_배송비포함_가격, Length: 164, dtype: int64


In [10]:
df.to_csv('./육회정리3.csv',encoding='utf-8-sig')  

In [12]:
print(total)

56
